In [2]:
import pathlib
import json

In [11]:
with open('../data/raw/via_2.0.8/via_powder_particle_masks.json', 'rb') as f:
    data = json.load(f)
root = data['_via_settings']['core']['default_filepath']

In [18]:
k = list(data['_via_img_metadata'].keys())

In [19]:
data['_via_img_metadata'][k[0]]

{'filename': 'Sc1Tile_001-001-000_0-000.png',
 'size': 1371223,
 'regions': [{'shape_attributes': {'name': 'polygon',
    'all_points_x': [702,
     686,
     686,
     692,
     698,
     703,
     714,
     725,
     743,
     781,
     813,
     820,
     813,
     793,
     769,
     755,
     748,
     741,
     739,
     728,
     717],
    'all_points_y': [487,
     514,
     539,
     554,
     566,
     575,
     585,
     592,
     597,
     593,
     563,
     531,
     494,
     475,
     465,
     462,
     457,
     457,
     463,
     468,
     473]},
   'region_attributes': {'Name': 'Powder',
    'Occluded?': {},
    'Contains_Satellite': 'Satellite',
    'Included Features': {'Satellite': True}}},
  {'shape_attributes': {'name': 'polygon',
    'all_points_x': [0,
     12,
     23,
     40,
     53,
     60,
     68,
     72,
     68,
     67,
     67,
     58,
     43,
     27,
     18,
     6,
     0],
    'all_points_y': [33,
     26,
     22,
     22,
     27,
     